In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import pprint
import json
from time import sleep
from datetime import date
import zipfile

# Incorporated citipy to determine city based on latitude and longitude
# pip install citipy
from citipy import citipy

# Output File location
output_folder = "output_data"

In [4]:
# Allows me to control # of rows for faster testing
max_rows = 1000 

zf = zipfile.ZipFile('../source_data/detail_listings.zip') 
df = pd.read_csv(zf.open('detail_listings.csv'), low_memory=False)

#df = pd.read_csv('../source_data/detail_listings.csv', low_memory=False)
df.head()

rows_in_df = len(df)
print(f'Data set has {rows_in_df} rows')

# Drop unneeded columns:
df = df.drop(columns = [
    "listing_url", 
    "thumbnail_url", 
    "picture_url", 
    "medium_url", 
    "xl_picture_url", 
    "host_url", 
    "host_thumbnail_url", 
    "host_picture_url", 
    "scrape_id",
    "host_listings_count", # There is a host_total_listings_count that is a dupe
    "host_acceptance_rate" # NaN
])
for col in df.columns: 
    print(col)
    if "_url" in col:
        print(col) 

df.head(10)

# if  rows_in_df > max_rows:
#    df = df.head(max_rows)
#    print(f'**** For testing purposes, removed rows. df now only has {max_rows}')

Data set has 23745 rows
id
last_scraped
name
summary
space
description
experiences_offered
neighborhood_overview
notes
transit
access
interaction
house_rules
host_id
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_is_superhost
host_neighbourhood
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
street
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
bed_type
amenities
square_feet
price
weekly_price
monthly_price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_nights_avg_ntm
maximum_nights_avg_ntm
calendar_updated
has_availability
availability_30
availability_60
availability_90
availability_365
c

id last_scraped                                              name  \
0  5065   2019-12-04                                          MAUKA BB   
1  5269   2019-12-05         Upcountry Hospitality in the 'Auwai Suite   
2  5387   2019-12-04               Hale Koa Studio & 1 Bedroom Units!!   
3  5389   2019-12-05                                     Keauhou Villa   
4  5390   2019-12-05                             STAY AT PRINCE KUHIO!   
5  5434   2019-12-04  Kauai  Ocean View 3 Story Tiki Tower North Shore   
6  5480   2019-12-04                Isle Of You Naturally Farm Retreat   
7  5930   2019-12-05                 Poipu Beach Guest House--- Orchid   
8  7896   2019-12-05                      Beachfront Maui Sunset Condo   
9  8833   2019-12-04                         Stay in the Heart of Kona   

                                             summary  \
0  Perfect for your vacation, Staycation or just ...   
1  The 'Auwai Suite is a lovely, self-contained a...   
2  This Wonderful Spacious Studio apt/flat is in ...   
3  It is less than 10 minute walk to the Keauhou ...   
4                                                NaN   
5  Hawaiian architecture , post and beam construc...   
6  The Best Choice for your Nudist Retreat. Where...   
7  Orchid is a first floor unit with  private bat...   
8  GREAT VALUE ON THE BEACH! Remodeled Condo, Tas...   
9  Our condo is located right across the street f...   

                                               space  \
0  Mauka B & B is 400 square feet studio with a p...   
1  The ‘Auwai Suite is a lovely, self-contained a...   
2  Hale Koa Duplex: This duplex has a wonderful, ...   
3  Location: Keauhou, Kona Coast, Big Island, Haw...   
4  Prince Kuhio, Studio Unit. Sleeps 2 privately....   
5  Our pristine and private 3 story, 3 bedroom, 2...   
6  Hey, want to stay someplace real for your next...   
7  From your front door, walk Southeast for just ...   
8  Sleeps up to 4 people. The Maui Sunset beachfr...   
9  Stay at the ROYAL KAHILI CONDOMINIUMS with a g...   

                                         description experiences_offered  \
0  Perfect for your vacation, Staycation or just ...                none   
1  The 'Auwai Suite is a lovely, self-contained a...                none   
2  This Wonderful Spacious Studio apt/flat is in ...                none   
3  It is less than 10 minute walk to the Keauhou ...                none   
4  Prince Kuhio, Studio Unit. Sleeps 2 privately....                none   
5  Hawaiian architecture , post and beam construc...                none   
6  The Best Choice for your Nudist Retreat. Where...                none   
7  Orchid is a first floor unit with  private bat...                none   
8  GREAT VALUE ON THE BEACH! Remodeled Condo, Tas...                none   
9  Our condo is located right across the street f...                none   

                               neighborhood_overview  \
0  Neighbors here are friendly but are not really...   
1  We are located on the "sunny side" of Waimea, ...   
2  IN a Farm belt area with small commercial farm...   
3  It is less than 10 minute walk to the Keauhou ...   
4                                                NaN   
5  The resort community of Princeville has many t...   
6  We are located on a rural one lane road going ...   
7  One of the best locations on Kauai,  on a smal...   
8  Conveniently located to all the sights on the ...   
9  Close to the heart of downtown Kona, but far e...   

                                               notes  \
0  Located near by is Kalopa State Park with 100 ...   
1                                                NaN   
2  tropical setting with view and easy access to ...   
3  It is less than 10 minute walk to the Keauhou ...   
4                                                NaN   
5  Hideaways beach is a beautiful scenic beach th...   
6                                                NaN   
7                    County of Kauai Permit TVR 4265  

In [26]:
df.describe()
print(len(df))

23745


In [ ]:
id
name
summary
space
description
experiences_offered
neighborhood_overview
notes
transit
access
interaction
house_rules
host_id
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_is_superhost
host_neighbourhood
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
street
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
bed_type
amenities
square_feet
price
weekly_price
monthly_price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_nights_avg_ntm
maximum_nights_avg_ntm
calendar_updated
has_availability
availability_30
availability_60
availability_90
availability_365
calendar_last_scraped
number_of_reviews
number_of_reviews_ltm
first_review
last_review
review_scores_rating
review_scores_accuracy
review_scores_cleanliness
review_scores_checkin
review_scores_communication
review_scores_location
review_scores_value
requires_license
license
jurisdiction_names
instant_bookable
is_business_travel_ready
cancellation_policy
require_guest_profile_picture
require_guest_phone_verification
calculated_host_listings_count
calculated_host_listings_count_entire_homes
calculated_host_listings_count_private_rooms
calculated_host_listings_count_shared_rooms
reviews_per_month

In [29]:
# CLEANUP 

# Drop rows with number_of_reviews = 0
# Get names of indexes for which column number_of_reviews has value 0
indexNames = df[df['number_of_reviews'] == 0 ].index
df.drop(indexNames , inplace=True)

# Drop rows with number_of_reviews = NaN
df.dropna(axis=0, subset=('number_of_reviews', ))

# Drop rows with NaN review data
df.dropna(axis=0, subset=(
    'review_scores_rating', 
    'review_scores_accuracy', 
    'review_scores_cleanliness', 
    'review_scores_checkin', 
    'review_scores_communication', 
    'review_scores_location', 
    'review_scores_value'
))

len(df)

18291

In [ ]:
host_group = # To do - switch this to groupby

In [17]:
# If a host has a hugh number of properties then their ratings are not higher 
#     than hosts with fewer properties
bins = [1, 60, 119, 179, 239, 299]

# labels - always one less than the range
group_names = ["0-59 properties", "60-119 properties", "120-179 properties", "180-239 properties", "240+ properties"]

df["Bin_NumProperties"] = pd.cut(df["host_total_listings_count"], bins, labels=group_names)
df_analyze = df[["host_id", "Bin_NumProperties", "host_total_listings_count"]]
df_analyze = df_analyze.rename(columns={"host_total_listings_count": "listings"})

# Loop through and get aggs:
for index, row in df_analyze.iterrows():
    df_analyze.loc[index, 'reviews'] = df_analyze["number_of_reviews"].sum()
    
df_analyze["reviews"] = 
df_analyze["rating_mean"] = df_analyze["review_scores_rating"].mean()
df_analyze["rating_value_mean"] = df_analyze["review_scores_value"].mean()
df_analyze.head()


KeyError: 'number_of_reviews'